# Notkun gervigreindar fyrir teikningu þrívíddarmynda

Nathan HK

In [2]:
import numpy as np
import os
import pywikibot
import sklearn as sk
from sklearn.model_selection import train_test_split
from stl import mesh
import time
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.set_default_device(device)
mappa = '/Users/002-nathan/Desktop/Envalys/envalys-nathan/'  # Change this

## Inngangsorð
Við ætlum að þjálfa gervigreindarlíkan til að teikna þrívíddarmyndir. Hvernig gerum við það?
- Fyrst reyndi ég að nota þrívíddardíla eða "þríla" (e. *voxels*). Ég taldi punktana í hverjum þríl, og þjálfaði líkan með tölunum, eins og líkan sem greinir ljósmyndir. GitHub: https://github.com/nholmesking/envalys-nathan/blob/4fd89a1515c1c3623a5b8cdf8c766858c465e451/Nathan%20HK.ipynb
  - Vandamál #1: Mac-GPU getur ekki neitt með þrívíddargögnum. Ég þurfti að nota CPU, sem er of hægt.
  - Vandamál #2: Hraðinn er $O(n^3)$.
  - Vandamál #3: Það var engin klár leið til að fara frá greiningu yfir á teikningu.
- Þá reyndi ég að nota líkanið 3DShape2VecSet, sem tekur lista yfir punkta og teinkir myndir sem eru eins og listinn.

Tilvísanir:
- https://arxiv.org/pdf/2301.11445
- https://github.com/1zb/3DShape2VecSet

Ég gerði litlar breytingar á upprunaforritið, því það er eitt forritasafn (e. *module*) sem virkar ekki lengur, og forritið var skrifað fyrir Windows (og ég nota Mac).

## Gögn
Gögnin frá 3DShape2VecSet eru 650 GB samtals; þetta er of mikið fyrir tölvuna mína.

Þessi gögn eru STL-skrár frá Wikimedia Commons. Það eru fimm flokkar:
- líkamshlutar
- byggingar
- rúmfræði
- geimfarartæki
- styttur

### Sækja gögn

In [3]:
flokkar = ['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
skrar = {}
catnum = {}

In [4]:
commons = pywikibot.Site('commons', 'commons')
cn = 0
for a in flokkar:
    print(a)
    cat = pywikibot.Category(commons, 'STL files of ' + a)
    catnum[a] = cn
    cn += 1
    n = 0
    for p in cat.members(member_type=['file']):
        if n % 10 == 0:
            print(n)
        mynd = pywikibot.FilePage(p)
        try:
            tempf = open(mappa + 'STLdata/' + a + '_' + p.title()[5:], 'r')
            tempf.close()
        except FileNotFoundError:
            mynd.download(filename=mappa + 'STLdata/' + a + '_' + p.title()[5:])
        try:
            skrar[a].append(p.title()[5:])
        except KeyError:
            skrar[a] = [p.title()[5:]]
        n += 1
        if n >= 100:
            break

body parts
0
10
20
30
40
50
60
70
80
buildings
0
10
20
geometric shapes
0
10
20
30
40
objects in space
0
10
20
30
40
50
sculptures
0
10
20
30
40
50


### Setja upp gögn fyrir notkun
Við eigum að breyta gögnunum úr STL-sniði í sniðið sem 3DShape2VecSet notar. Þetta snið er með lista yfir punkta eftir hvort þeir séu innan á forminu eða utan, en gögnin frá Wikimedia Commons er ekki með það.

Með ```isInside()``` getum útreiknað hvort lína og þríhyrningur mætast, og í hvora átt. Við búum til tvær línur sem fara beint í X-, Y-, eða Z-áttina frá punktnum (eina línu upp og eina línu niður), og teljum þríhyrningana sem línan mætir; ef talan er slétt, þá er punkturinn utan á forminu, og ef talan er oddatala, þá er punkturinn innan. En það tekur of langan tíma; sumar myndir eru með fleiri en milljón þríhyrninga. 

Þess vegna bjó ég til þetta reknirit: Við deilum myndinni í 32x32 eða fleiri ílát í þrem ílátahópum, einum fyrir hverja vídd, eftir hnitum hinna vídda. Í byrjun setjum við alla þríhyrninga í þeirra ílát; sumir þríhyrningar, sem eru á línunni milli íláta, eru settir í tvö eða fleiri ílát. Fyrir hvern listapunkt eigum við bara að leita í rétta ílátinu, í hópnum þar sem þríhyrningafjöldinn er lægstur.

Með þessu reikniriti (og litlum öðrum breytingum) er sniðbreytingin **400** sinnum fljótari en hún var þegar ég byrjaði. Ég skrifaði sjálfur allt hérna nema ```isInside()```; sjáðu kóðann fyrir tilvísun.

In [52]:
def isInBB(a, randpoint, att):
    if att == 'x':
        if (a[1] > randpoint[1] and a[4] > randpoint[1] and a[7] > randpoint[1]):
            return False
        if (a[1] < randpoint[1] and a[4] < randpoint[1] and a[7] < randpoint[1]):
            return False
        if (a[2] > randpoint[2] and a[5] > randpoint[2] and a[8] > randpoint[2]):
            return False
        if (a[2] < randpoint[2] and a[5] < randpoint[2] and a[8] < randpoint[2]):
            return False
        return True
    elif att == 'y':
        if (a[0] > randpoint[0] and a[3] > randpoint[0] and a[6] > randpoint[0]):
            return False
        if (a[0] < randpoint[0] and a[3] < randpoint[0] and a[6] < randpoint[0]):
            return False
        if (a[2] > randpoint[2] and a[5] > randpoint[2] and a[8] > randpoint[2]):
            return False
        if (a[2] < randpoint[2] and a[5] < randpoint[2] and a[8] < randpoint[2]):
            return False
        return True
    elif att == 'z':
        if (a[0] > randpoint[0] and a[3] > randpoint[0] and a[6] > randpoint[0]):
            return False
        if (a[0] < randpoint[0] and a[3] < randpoint[0] and a[6] < randpoint[0]):
            return False
        if (a[1] > randpoint[1] and a[4] > randpoint[1] and a[7] > randpoint[1]):
            return False
        if (a[1] < randpoint[1] and a[4] < randpoint[1] and a[7] < randpoint[1]):
            return False
        return True

In [53]:
def isInside(a, n_, randpoint, att):
    if not np.any(n_):
        return 0
 
    # This part is from https://diegoinacio.github.io/creative-coding-notebooks-page/pages/ray-intersection_triangle.html
    if att == 'x':
        e = np.array([1, 0, 0])     # Ray direction
    elif att == 'y':
        e = np.array([0, 1, 0])     # Ray direction
    elif att == 'z':
        e = np.array([0, 0, 1])     # Ray direction
    if not np.any(np.dot(n_, e)):
        return 0
    d = - np.dot(n_, a[0:3])
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return 1
        
    if att == 'x':
        e = np.array([-1, 0, 0])     # Ray direction
    elif att == 'y':
        e = np.array([0, -1, 0])     # Ray direction
    elif att == 'z':
        e = np.array([0, 0, -1])     # Ray direction
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return -1
    return 0

In [54]:
for cat in skrar:
    tts = []
    print(cat)
    byrjun = time.time()
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '"')
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '"')
    for fi in skrar[cat]:
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        utskra_mesh = open(mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '/' + fi + '.npz', 'wb')
        np.savez(utskra_mesh, points=np.array(gogn.v0, dtype=np.float32), scale=1)
        tts.append(fi)
    train, val = train_test_split(tts)
    trainlst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/train.lst', 'w')
    print('train')
    for fi in train:
        print(fi)
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        afora = ([a[0] for a in gogn.v0], [a[1] for a in gogn.v0], [a[2] for a in gogn.v0])
        mm = ((min(afora[0]), max(afora[0])),
              (min(afora[1]), max(afora[1])),
              (min(afora[2]), max(afora[2])))
        num_ilat = max(32, int((gogn.points.shape[0] / 500) ** (1/2)))
        ilat_X = []
        for i in range(num_ilat):
            ilat_X.append([])
            for j in range(num_ilat):
                ilat_X[i].append([])
        ilat_Y = []
        for i in range(num_ilat):
            ilat_Y.append([])
            for j in range(num_ilat):
                ilat_Y[i].append([])
        ilat_Z = []
        for i in range(num_ilat):
            ilat_Z.append([])
            for j in range(num_ilat):
                ilat_Z[i].append([])
        nyfirm = (num_ilat / (mm[0][1] - mm[0][0]), num_ilat / (mm[1][1] - mm[1][0]), num_ilat / (mm[2][1] - mm[2][0]))
        ilatid = lambda x, y, z: (max(0, min(int((x - mm[0][0]) * nyfirm[0]), num_ilat - 1)),
                                  max(0, min(int((y - mm[1][0]) * nyfirm[1]), num_ilat - 1)),
                                  max(0, min(int((z - mm[2][0]) * nyfirm[2]), num_ilat - 1)))
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        for h in range(gogn.points.shape[0]):
            a = gogn.points[h]
            minid = ilatid(min(a[0], a[3], a[6]), min(a[1], a[4], a[7]), min(a[2], a[5], a[8]))
            maxid = ilatid(max(a[0], a[3], a[6]), max(a[1], a[4], a[7]), max(a[2], a[5], a[8]))
            for i in range(minid[0], maxid[0] + 1):
                for j in range(minid[1], maxid[1] + 1):
                    ilat_Z[i][j].append(h)
            for i in range(minid[0], maxid[0] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_Y[i][k].append(h)
            for j in range(minid[1], maxid[1] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_X[j][k].append(h)
        n_dat = {}
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            fata = ilatid(randpoint[0], randpoint[1], randpoint[2])
            pct = [0, 0]
            if len(ilat_X[fata[1]][fata[2]]) <= min(len(ilat_Y[fata[0]][fata[2]]), len(ilat_Z[fata[0]][fata[1]])):
                for h in ilat_X[fata[1]][fata[2]]:
                    if isInBB(gogn.points[h], randpoint, 'x'):
                        try:
                            n_dat[h]
                        except KeyError:
                            a = gogn.points[h]
                            AB = a[3:6] - a[0:3]               # Oriented segment A to B
                            AC = a[6:9] - a[0:3]               # Oriented segment A to C
                            n = np.cross(AB, AC)     # Normal vector
                            n_ = n/np.linalg.norm(n) # Normalized normal
                            n_dat[h] = n_
                        isi = isInside(gogn.points[h], n_dat[h], randpoint, 'x')
                        if isi == 1:
                            pct[1] += 1
                        elif isi == -1:
                            pct[0] += 1
            elif len(ilat_Y[fata[0]][fata[2]]) <= min(len(ilat_X[fata[1]][fata[2]]), len(ilat_Z[fata[0]][fata[1]])):
                for h in ilat_Y[fata[0]][fata[2]]:
                    if isInBB(gogn.points[h], randpoint, 'y'):
                        try:
                            n_dat[h]
                        except KeyError:
                            a = gogn.points[h]
                            AB = a[3:6] - a[0:3]               # Oriented segment A to B
                            AC = a[6:9] - a[0:3]               # Oriented segment A to C
                            n = np.cross(AB, AC)     # Normal vector
                            n_ = n/np.linalg.norm(n) # Normalized normal
                            n_dat[h] = n_
                        isi = isInside(gogn.points[h], n_dat[h], randpoint, 'y')
                        if isi == 1:
                            pct[1] += 1
                        elif isi == -1:
                            pct[0] += 1
            elif len(ilat_Z[fata[0]][fata[1]]) <= min(len(ilat_X[fata[1]][fata[2]]), len(ilat_Y[fata[0]][fata[2]])):
                for h in ilat_Z[fata[0]][fata[1]]:
                    if isInBB(gogn.points[h], randpoint, 'z'):
                        try:
                            n_dat[h]
                        except KeyError:
                            a = gogn.points[h]
                            AB = a[3:6] - a[0:3]               # Oriented segment A to B
                            AC = a[6:9] - a[0:3]               # Oriented segment A to C
                            n = np.cross(AB, AC)     # Normal vector
                            n_ = n/np.linalg.norm(n) # Normalized normal
                            n_dat[h] = n_
                        isi = isInside(gogn.points[h], n_dat[h], randpoint, 'z')
                        if isi == 1:
                            pct[1] += 1
                        elif isi == -1:
                            pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                vol_points.append(randpoint)
                vol_label.append(pct[0] % 2)
        print('train vol', time.time() - byrjun)
        near_points = []
        near_label = []
        for i in range(4096):
            gp = gogn.points[np.random.randint(0, gogn.points.shape[0])]
            randpoint = (gp[0] + np.random.uniform(mm[0][0] / 100, mm[0][1] / 100),
                         gp[1] + np.random.uniform(mm[1][0] / 100, mm[1][1] / 100),
                         gp[2] + np.random.uniform(mm[2][0] / 100, mm[2][1] / 100))
            # Is point inside or outside shape?
            fata = ilatid(randpoint[0], randpoint[1], randpoint[2])
            pct = [0, 0]
            if len(ilat_X[fata[1]][fata[2]]) <= min(len(ilat_Y[fata[0]][fata[2]]), len(ilat_Z[fata[0]][fata[1]])):
                for h in ilat_X[fata[1]][fata[2]]:
                    if isInBB(gogn.points[h], randpoint, 'x'):
                        try:
                            n_dat[h]
                        except KeyError:
                            a = gogn.points[h]
                            AB = a[3:6] - a[0:3]               # Oriented segment A to B
                            AC = a[6:9] - a[0:3]               # Oriented segment A to C
                            n = np.cross(AB, AC)     # Normal vector
                            n_ = n/np.linalg.norm(n) # Normalized normal
                            n_dat[h] = n_
                        isi = isInside(gogn.points[h], n_dat[h], randpoint, 'x')
                        if isi == 1:
                            pct[1] += 1
                        elif isi == -1:
                            pct[0] += 1
            elif len(ilat_Y[fata[0]][fata[2]]) <= min(len(ilat_X[fata[1]][fata[2]]), len(ilat_Z[fata[0]][fata[1]])):
                for h in ilat_Y[fata[0]][fata[2]]:
                    if isInBB(gogn.points[h], randpoint, 'y'):
                        try:
                            n_dat[h]
                        except KeyError:
                            a = gogn.points[h]
                            AB = a[3:6] - a[0:3]               # Oriented segment A to B
                            AC = a[6:9] - a[0:3]               # Oriented segment A to C
                            n = np.cross(AB, AC)     # Normal vector
                            n_ = n/np.linalg.norm(n) # Normalized normal
                            n_dat[h] = n_
                        isi = isInside(gogn.points[h], n_dat[h], randpoint, 'y')
                        if isi == 1:
                            pct[1] += 1
                        elif isi == -1:
                            pct[0] += 1
            elif len(ilat_Z[fata[0]][fata[1]]) <= min(len(ilat_X[fata[1]][fata[2]]), len(ilat_Y[fata[0]][fata[2]])):
                for h in ilat_Z[fata[0]][fata[1]]:
                    if isInBB(gogn.points[h], randpoint, 'z'):
                        try:
                            n_dat[h]
                        except KeyError:
                            a = gogn.points[h]
                            AB = a[3:6] - a[0:3]               # Oriented segment A to B
                            AC = a[6:9] - a[0:3]               # Oriented segment A to C
                            n = np.cross(AB, AC)     # Normal vector
                            n_ = n/np.linalg.norm(n) # Normalized normal
                            n_dat[h] = n_
                        isi = isInside(gogn.points[h], n_dat[h], randpoint, 'z')
                        if isi == 1:
                            pct[1] += 1
                        elif isi == -1:
                            pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                near_points.append(randpoint)
                near_label.append(pct[0] % 2)
        print('train near', time.time() - byrjun)
        if len(vol_points) > 0 and len(near_points) > 0:
            trainlst.write(fi + '.npz\n')
            np.savez(utskra,
                     vol_points=np.array(vol_points, dtype=np.float32),
                     vol_label=np.array(vol_label),
                     near_points=np.array(near_points, dtype=np.float32),
                     near_label=np.array(near_label))
    trainlst.close()
    vallst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/val.lst', 'w')
    print('val')
    for fi in val:
        print(fi)
        vallst.write(fi + '.npz\n')
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        num_ilat = max(32, int((gogn.points.shape[0] / 500) ** (1/2)))
        ilat_X = []
        for i in range(num_ilat):
            ilat_X.append([])
            for j in range(num_ilat):
                ilat_X[i].append([])
        ilat_Y = []
        for i in range(num_ilat):
            ilat_Y.append([])
            for j in range(num_ilat):
                ilat_Y[i].append([])
        ilat_Z = []
        for i in range(num_ilat):
            ilat_Z.append([])
            for j in range(num_ilat):
                ilat_Z[i].append([])
        nyfirm = (num_ilat / (mm[0][1] - mm[0][0]), num_ilat / (mm[1][1] - mm[1][0]), num_ilat / (mm[2][1] - mm[2][0]))
        ilatid = lambda x, y, z: (max(0, min(int((x - mm[0][0]) * nyfirm[0]), num_ilat - 1)),
                                  max(0, min(int((y - mm[1][0]) * nyfirm[1]), num_ilat - 1)),
                                  max(0, min(int((z - mm[2][0]) * nyfirm[2]), num_ilat - 1)))
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        for h in range(gogn.points.shape[0]):
            a = gogn.points[h]
            minid = ilatid(min(a[0], a[3], a[6]), min(a[1], a[4], a[7]), min(a[2], a[5], a[8]))
            maxid = ilatid(max(a[0], a[3], a[6]), max(a[1], a[4], a[7]), max(a[2], a[5], a[8]))
            for i in range(minid[0], maxid[0] + 1):
                for j in range(minid[1], maxid[1] + 1):
                    ilat_Z[i][j].append(h)
            for i in range(minid[0], maxid[0] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_Y[i][k].append(h)
            for j in range(minid[1], maxid[1] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_X[j][k].append(h)
        n_dat = {}
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            fata = ilatid(randpoint[0], randpoint[1], randpoint[2])
            pct = [0, 0]
            if len(ilat_X[fata[1]][fata[2]]) <= min(len(ilat_Y[fata[0]][fata[2]]), len(ilat_Z[fata[0]][fata[1]])):
                for h in ilat_X[fata[1]][fata[2]]:
                    if isInBB(gogn.points[h], randpoint, 'x'):
                        try:
                            n_dat[h]
                        except KeyError:
                            a = gogn.points[h]
                            AB = a[3:6] - a[0:3]               # Oriented segment A to B
                            AC = a[6:9] - a[0:3]               # Oriented segment A to C
                            n = np.cross(AB, AC)     # Normal vector
                            n_ = n/np.linalg.norm(n) # Normalized normal
                            n_dat[h] = n_
                        isi = isInside(gogn.points[h], n_dat[h], randpoint, 'x')
                        if isi == 1:
                            pct[1] += 1
                        elif isi == -1:
                            pct[0] += 1
            elif len(ilat_Y[fata[0]][fata[2]]) <= min(len(ilat_X[fata[1]][fata[2]]), len(ilat_Z[fata[0]][fata[1]])):
                for h in ilat_Y[fata[0]][fata[2]]:
                    if isInBB(gogn.points[h], randpoint, 'y'):
                        try:
                            n_dat[h]
                        except KeyError:
                            a = gogn.points[h]
                            AB = a[3:6] - a[0:3]               # Oriented segment A to B
                            AC = a[6:9] - a[0:3]               # Oriented segment A to C
                            n = np.cross(AB, AC)     # Normal vector
                            n_ = n/np.linalg.norm(n) # Normalized normal
                            n_dat[h] = n_
                        isi = isInside(gogn.points[h], n_dat[h], randpoint, 'y')
                        if isi == 1:
                            pct[1] += 1
                        elif isi == -1:
                            pct[0] += 1
            elif len(ilat_Z[fata[0]][fata[1]]) <= min(len(ilat_X[fata[1]][fata[2]]), len(ilat_Y[fata[0]][fata[2]])):
                for h in ilat_Z[fata[0]][fata[1]]:
                    if isInBB(gogn.points[h], randpoint, 'z'):
                        try:
                            n_dat[h]
                        except KeyError:
                            a = gogn.points[h]
                            AB = a[3:6] - a[0:3]               # Oriented segment A to B
                            AC = a[6:9] - a[0:3]               # Oriented segment A to C
                            n = np.cross(AB, AC)     # Normal vector
                            n_ = n/np.linalg.norm(n) # Normalized normal
                            n_dat[h] = n_
                        isi = isInside(gogn.points[h], n_dat[h], randpoint, 'z')
                        if isi == 1:
                            pct[1] += 1
                        elif isi == -1:
                            pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                vol_points.append(randpoint)
                vol_label.append(pct[0] % 2)
        if len(vol_points) > 0:
            print('val vol', time.time() - byrjun)
            np.savez(utskra,
                     vol_points=np.array(vol_points, dtype=np.float32),
                     vol_label=np.array(vol_label))
    vallst.close()
    print(time.time() - byrjun)
    print('----')

body parts


mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/body parts: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/body parts: File exists


train
Vh-f-mammary-gland-r.stl
585330
train vol 8.585448265075684
train near 23.77039098739624
Dödsmask av Ivar Arosenius NMSk2348b - Nationalmuseum -3724438ee23c4defadf93d833e8c870d.stl
60000
train vol 25.43987202644348
train near 28.31662106513977
We Are Beautiful – 554268-FSAN-1 – solid.stl
423450
train vol 34.15127396583557
train near 38.57626533508301
We Are Beautiful – 507391-OSNN-1 – surface.stl
107777
train vol 40.4840202331543
train near 42.263286113739014
We Are Beautiful – 591522-FSAN-1 – solid.stl
592592
train vol 50.02584719657898
train near 54.94574213027954
Vh-f-liver.stl
93303
train vol 58.68580603599548
train near 64.9116883277893
We Are Beautiful – 768772-LSNN-1 – surface.stl
478890
train vol 71.39727830886841
train near 75.35963320732117
Vh-m-kidney-r.stl
84088
train vol 80.26779794692993
train near 92.39732503890991
3D ear.stl
222732
train vol 96.33809208869934
train near 100.2714192867279
Vh-f-ovary-l.stl
424
train vol 101.13611125946045
train near 102.903330087661

KeyboardInterrupt: 

Sniðbreytingin tók 28 mínútur.

## Líkan
Líkanið þarf ekki að búa til góðar myndir. Það þarf bara að virka án villna, sem sýnir okkur að við þurfum bara meiri og betri gögn, því vísindagreinin er með góðar myndir.

In [18]:
os.system('cd ' + mappa)
os.system('torchrun --nproc_per_node=4 3DShape2VecSet/main_ae.py --accum_iter=2 --model ae_d512_m512 '
          '--data_path 3DShape2VecSet/data --output_dir output/ae/ae_d512_m512 --log_dir output/ae/ae_d512_m512 '
          '--num_workers 10 --point_cloud_size 2048 --batch_size 64 --epochs 200 --warmup_epochs 5')

W0825 09:05:21.680789 8286703424 torch/distributed/elastic/multiprocessing/redirects.py:28] NOTE: Redirects are currently not supported in Windows or MacOs.
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] 
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] *****************************************
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] *****************************************


['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']

['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
Model = AutoEncoder(
  (cross_attend_blocks): ModuleList(
    (0): PreNorm(
      (fn): Attention(
        (to_q): Linear(in_features=512, out_features=512, bias=False)
        (to_kv): Linear(in_features=512, out_features=1024, bias=False)
        (to_out): Linear(in_features=512, out_features=512, bias=True)
        (drop_path): Identity()


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Epoch: [0]  [ 0/47]  eta: 0:43:45  lr: 0.000000  loss: 0.7250 (0.7250)  loss_vol: 0.6592 (0.6592)  loss_near: 0.6580 (0.6580)  iou: 0.0139 (0.0139)  time: 55.8586  data: 19.8939
Epoch: [0]  [ 0/47]  eta: 0:49:22  lr: 0.000000  loss: 0.7185 (0.7185)  loss_vol: 0.6532 (0.6532)  loss_near: 0.6534 (0.6534)  iou: 0.0089 (0.0089)  time: 63.0310  data: 19.9302
Epoch: [0]  [ 0/47]  eta: 0:50:54  lr: 0.000000  loss: 0.7204 (0.7204)  loss_vol: 0.6549 (0.6549)  loss_near: 0.6550 (0.6550)  iou: 0.0234 (0.0234)  time: 64.9863  data: 16.6555
Epoch: [0]  [ 0/47]  eta: 1:23:32  lr: 0.000000  loss: 0.7272 (0.7272)  loss_vol: 0.6611 (0.6611)  loss_near: 0.6610 (0.6610)  iou: 0.0238 (0.0238)  time: 106.6386  data: 16.7561


W0825 09:35:12.375863 8286703424 torch/distributed/elastic/agent/server/api.py:688] Received Signals.SIGINT death signal, shutting down workers
W0825 09:35:12.392325 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40348 closing signal SIGINT
W0825 09:35:12.393383 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40349 closing signal SIGINT
W0825 09:35:12.393764 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40350 closing signal SIGINT
W0825 09:35:12.398679 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40351 closing signal SIGINT
W0825 09:35:42.408644 8286703424 torch/distributed/elastic/multiprocessing/api.py:875] Unable to shutdown process 40348 via Signals.SIGINT, forcefully exiting via Signals.SIGKILL
W0825 09:35:49.303166 8286703424 torch/distributed/elastic/multiprocessing/api.py:875] Unable to shutdown process 40349 via Signals.SIGINT, forcefully ex

256

Það eru 200 lotna (e. *epochs*), og hver lota tekur klukkustund; við eigum ekki nógan tíma fyrir alla þjálfunina. **En hún er hafin án villna.**

# Lokaorð
Hvert förum við héðan? 3DShape2VecSet er með margar takmarkanir.
- Það kann bara að búa til einfaldar myndir, eins og "stóll". Með greiningarlíkani getum við mögulega búið til fjölfaldar myndir. Ef notandi segir "stóll með kodda", þá býr líkanið okkar til mynd af stóli og mynd af kodda, sem líkanið sameinar.
- Það skilur ekki mannamál.
- Það er ekki með mikil gögn. Með mínu forriti hérna getum við búið til eins mikil gögn og við viljum; við getum tekið myndir frá notendum og búum til lista yfir punkta, og notað þá til að þjálfa líkanið.

Ég er viss um að, með meiri gögn og betri tölvu, við getum að minnsta kosti eitt af þessum.